## Modul

Modul yang digunakan dalam script ini dapat dilihat pada cell berikut. Jika terdapat pesan error bahwa modul tidak tersedia, dapat dilakukan instalasi terlebih dahulu dengan menggunakan `conda`

```{python}
!conda install <module>
```

Atau bisa menggunakan `pip`

```{python}
!pip install <module>
```

Instalasi menggunakan `pip` hendaknya dilakukan jika modul tidak tersedia dalam repository `conda`.

In [1]:
import os
import sys
import json
import time
import base64
from IPython.core.display import HTML

from getpass import getpass

import requests
import warnings
warnings.filterwarnings('ignore')

import hda

import xarray as xr
import numpy as np

from rich.jupyter import print

import cartopy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs



In [2]:
download_dir = os.path.join(os.path.expanduser('~'),"edskywalker","imber-sen3", "downloaded")

os.makedirs(download_dir, exist_ok=True)

print(download_dir)

/home/jovyan/edskywalker/imber-sen3/downloaded

## Input `username` dan `password` 

Untuk mengakses data WEKEO, Anda perlu memiliki akun terlebih dahulu. Silakan pelajari mengenai pembuatan akun terlebih dahulu di [sini](https://help.wekeo.eu/en/articles/9389186-how-to-create-a-wekeo-account). 

In [3]:
user = r'edskywalker' #input("Enter your name: ")
passw = r'padangLua103#' #getpass("Enter your password: ")

c = hda.Client(hda.Configuration(user=user, password=passw))

## Input *Data Query*

Masukkan parameter terkait dengan jenis data yang diinginkan serta lokasi dan waktunya.

`Dataset` adalah dataset-id. Value yang tepat bisa ditemukan di 

Untuk data Sentinel-3 OLCI, yang tersedia adalah sebagai berikut:

- `EO:EUM:DAT:SENTINEL-3:0556` &rarr; **2016-04-25** -- **28-04-2021**
- `EO:EUM:DAT:SENTINEL-3:OL_2_WFR___` &rarr; **2021-04-29** -- **2023-12-31**

Sentinel-3 juga tersedia dalam 2 jenis satelit, `Sentinel-3A` dan `Sentinel-3B`. Jika pilihan ini tidak diisi, pada `query` akan terpilih semua satelit. Perlu dicatat bahwa Sentinel-3B tersedia mulai tanggal **2018-05-18**.

In [4]:
query = {
  "dataset_id": "EO:EUM:DAT:SENTINEL-3:0556", #input("Dataset: "), 
  "dtstart": "2017-04-25T00:00:00.000Z", #input("Time start: "),
  "dtend": "2018-04-28T00:00:00.000Z", #input("Time end: "),
  "bbox": [
    106.33442417084652, #float(input("West bound: ")), #106.33442417084652,
    -6.226519426392115, #float(input("South bound: ")), #-6.226519426392115,
    107.12675355792621, #float(input("East bound: ")), #107.12675355792621,
    -5.321432926835288 #float(input("North bound: ")), #-5.321432926835288
  ],
  "sat": "Sentinel-3A",# input("Satellite: "), #"Sentinel-3B",
  "type": "OL_2_WFR___",
  "timeliness": "NT",
  "itemsPerPage": 200,
  "startIndex": 0
}


In [5]:
search_result = c.search(query)
print(search_result)


SearchResults[items=217,volume=54.9GB]

In [6]:
data_id = search_result[0].results[0]['id']

print(data_id)

search_result[0].download()

S3A_OL_2_WFR____20180427T021754_20180427T022054_20211130T163312_0179_030_274______MAR_R_NT_003.SEN3

In [7]:
import zipfile

with zipfile.ZipFile(data_id + '.zip', 'r') as zip_ref:
    zip_ref.extractall(download_dir)
    print(f'Unzipping of product {data_id} finished.')
    os.remove(data_id + '.zip')

Unzipping of product 
S3A_OL_2_WFR____20180427T021754_20180427T022054_20211130T163312_0179_030_274______MAR_R_NT_003.SEN3 finished.

In [8]:
safe_dir = os.path.join(download_dir,data_id)

geo_coords = xr.open_dataset(os.path.join(download_dir,data_id,'geo_coordinates.nc'))
flag_file = xr.open_dataset(os.path.join(download_dir,data_id,'wqsf.nc'))
chl_file = xr.open_dataset(os.path.join(download_dir,data_id,'chl_nn.nc'))

In [9]:
lon = geo_coords.variables['longitude'].data
lat = geo_coords.variables['latitude'].data
flags_data = flag_file.variables['WQSF'].data
chl_nn = chl_file.variables['CHL_NN'].data

In [10]:
def flag_data_fast(list_flag, flag_names, flag_values, flag_data, flag_type='WQSF'):
    flag_bits = np.uint64()
    if flag_type == 'SST':
        flag_bits = np.uint8()
    elif flag_type == 'WQSF_lsb':
        flag_bits = np.uint32()
    for flag in list_flag:
        try:
            flag_bits = flag_bits | flag_values[flag_names.index(flag)]
        except:
            print(flag + 'not present')
    return (flag_data & flag_bits) > 0			


In [11]:
list_flag = ['LAND', 'CLOUD', 'CLOUD_AMBIGUOUS', 'CLOUD_MARGIN', 'INVALID', 'COSMETIC', 'SATURATED', 'SUSPECT', 'HISOLZEN', 'HIGHGLINT', 'SNOW_ICE', 'AC_FAIL', 'WHITECAPS', 'ADJAC', 'RWNEG_O2', 'RWNEG_O3', 'RWNEG_O4', 'RWNEG_O5', 'RWNEG_O6', 'RWNEG_O7', 'RWNEG_O8', 'OCNN_FAIL']

flag_names = flag_file['WQSF'].flag_meanings.split(' ') #flag names
flag_vals = flag_file['WQSF'].flag_masks #flag bit values
flags_data = flag_file.variables['WQSF'].data

flag_mask = flag_data_fast(list_flag, flag_names, flag_vals, flags_data, flag_type='WQSF')

flag_mask = flag_mask.astype(float)
flag_mask[flag_mask == 0] = np.nan

chl_flagged = chl_nn * flag_mask

In [12]:
geo_coords.close()
chl_file.close()
flag_file.close()

print('Lon shape: ', lon.shape)
print('Mask shape; ', flag_mask.shape)
print('Flagged shape: ', chl_flagged.shape)

Lon shape: 
(4091, 4865)

Mask shape; 
(4091, 4865)

Flagged shape: 
(4091, 4865)

In [13]:
# print(chl_file)

dta = xr.Dataset()

dta['lon'] = xr.DataArray(lon, dims=('x','y'))
dta['lat'] = xr.DataArray(lat, dims=('x','y'))
dta['mask'] = xr.DataArray(flag_mask, dims=('x','y'))
dta['chl_nn'] = xr.DataArray(chl_flagged, dims=('x','y'))

dta = dta.set_coords(['lon','lat'])

print(dta)

<xarray.Dataset> Size: 637MB
Dimensions:  (x: 4091, y: 4865)
Coordinates:
    lon      (x, y) float64 159MB 106.7 106.8 106.8 106.8 ... 115.9 116.0 116.0
    lat      (x, y) float64 159MB 0.3533 0.3528 0.3522 ... -12.86 -12.86 -12.86
Dimensions without coordinates: x, y
Data variables:
    mask     (x, y) float64 159MB 1.0 1.0 1.0 1.0 1.0 ... 1.0 1.0 1.0 1.0 1.0
    chl_nn   (x, y) float64 159MB nan nan nan nan nan ... nan nan nan nan nan

In [ ]:
fig, ax = plt.subplots(figsize=[12,6], ncols=2, layout='constrained', subplot_kw=dict(projection=ccrs.Mercator()))

# ax.add_feature(cartopy.feature.LAND.with_scale('10m'), facecolor = 'beige', edgecolor='black', zorder = 1)
for i in range(2):
    
    ax[i].coastlines()
    ini = ax[i].gridlines(draw_labels = True, alpha=0.5)
    ini.top_labels = False
    ini.right_labels = False

dta['mask'].plot(ax=ax[0], x='lon',y='lat', vmin=0, vmax=1, transform=ccrs.PlateCarree(), zorder=0)
dta['chl_nn'].plot(ax=ax[1], x='lon',y='lat', vmin=np.log10(0.01), vmax=np.log10(20), transform=ccrs.PlateCarree(), zorder=0)

